# Project_S5 Time Difference Of Arrival (TDOA)
###### von Gerrit Schröder und Nils Müller

### Installieren der benötigten Bibliotheken

In [1]:
!pip install ipyleaflet
!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


### Einbinden der benötigten Bibliotheken

In [2]:
from ipyleaflet import Map, DrawControl, CircleMarker, Polyline, Marker, Circle #OSM Kartendarstellung
import geocoder as geo                                 #Addresse zu Koordinaten
import numpy as np                                     #Mathematische Operationen
import matplotlib.pyplot as plt                        #Plotten
import scipy.constants as const                        #Naturkonstanten
%matplotlib notebook

### Empfänger

In [3]:
#Addressen der Empfänger
RX_addr = ["Neustadtswall 30, 28199 Bremen",            #Hochschule Bremen 
          "Konsul Smidt Straße 90, 28217 Bremen",       #Landmark Tower
          "Flughafenallee 10, 28199 Bremen",            #ZIMT
          "Falkenberger-Straße 1, 28215 Bremen",        #Punkt 4
          "Senator-Helmken-Straße 3, 28197 Bremen"]     #Punkt 5
RX_init = ["AB", "LM", "AP", "P4", "P5"]

RX_coord = np.zeros((len(RX_addr), 2))                  #Nullmatrix erstellen

for i in range(0, len(RX_addr)):                        #Koordinatenzuweisung
    RX = geo.osm(RX_addr[i])                            #Beziehe die Daten von OpenStreetMap für die Adressen
    RX_coord[i, 0] = RX.lat                             #Speichere Längengrad
    RX_coord[i, 1] = RX.lng                             #Speichere Breitengrad
RX_coord

array([[ 53.0726811 ,   8.7945873 ],
       [ 53.0975158 ,   8.76017766],
       [ 53.0555413 ,   8.7840737 ],
       [ 53.090774  ,   8.80498725],
       [ 53.0809646 ,   8.7537748 ]])

In [4]:
RXcoord1=np.zeros((len(RX_addr), 2))
RXcoord1[0,:]=[53.072640, 8.793820]
RXcoord1[1,:]=[53.097637, 8.760266]
RXcoord1[2,:]=[53.055366, 8.783361]
RXcoord1[3,:]=[53.090756, 8.804930]
RXcoord1[4,:]=[53.080220, 8.750380]
RX_coord =RXcoord1
RX_coord

array([[ 53.07264 ,   8.79382 ],
       [ 53.097637,   8.760266],
       [ 53.055366,   8.783361],
       [ 53.090756,   8.80493 ],
       [ 53.08022 ,   8.75038 ]])

### Referenzpunkt

In [5]:
#Addresse Referenzpunkt
REF_addr = "Utbremer Str. 90, 28217 Bremen"             #Fernsehturm Bremen Walle
REF = geo.osm(REF_addr)                                 #Beziehe die Daten von OpenStreetMap für die Adressen
REF_coord = np.zeros((1, 2))                            #Nullvektor erstellen
REF_coord = [REF.lat, REF.lng]                          #Längen und Breitengrad speichern
REF_coord

[53.0950487, 8.7925119]

### Sender (wird nur zur Überprüfung des Programms benötigt)

In [6]:
#Adresse Signal
SIGN_addr = "Johann-Bornemacher Straße 4, 28217 Bremen" #Addresse des Senders
SIGN = geo.osm(SIGN_addr)                               #Beziehe die Daten von OpenStreetMap für die Adressen
SIGN_coord = np.zeros((1, 2))                           #Nullmatrix erstellen
SIGN_coord = [SIGN.lat, SIGN.lng]                       #Längen und Breitengrad speichern
SIGN_coord

[53.0915631, 8.7901959]

### Berechnung der TDOA Werte für den simulierten Sender

In [7]:
#Berechnung theoretische TDOA Werte
RX_coord_rad = np.radians(RX_coord)                     #Transformation in Gradmaß
SIGN_coord_rad = np.radians(SIGN_coord)                 #Transformation in Gradmaß
distance = np.zeros(len(RX_addr))                       #Nullvektor erstellen

def dist(RX_coord_rad, SIGN_coord_rad):                 #Funktion, Abstand Empfänger/ Sender
    earth_radius = 6371                                 #Radius Erde in Bremen in km
    
    for j in range(0, len(RX_coord)):                   #Berechnung mithilfe der "Haversine - Formel"
        delta_lat = RX_coord_rad[j, 0] - SIGN_coord_rad[0]
        delta_lng = RX_coord_rad[j, 1] - SIGN_coord_rad[1] 
        a = np.sin(delta_lat/2)**2 + np.cos(RX_coord_rad[j,0]) * np.cos(SIGN_coord_rad[0]) * np.sin(delta_lng/2)**2   # haversine formula 
        c = 2 * np.arcsin(np.sqrt(a))
        distance[j] = c * earth_radius
    return distance

In [8]:
dis = dist(RX_coord_rad, SIGN_coord_rad)                       #Distanz funkion aufrufen
delta_x = np.zeros(len(RX_addr))                               #Nullvektor aufstellen
for i in range(0, len(RX_coord)-1):                          #Berechnen der simulierten Laufwegunterschiede
    delta_x[i] = (dis[i] - dis[i+1]) * 1000
delta_x[len(RX_coord) - 1] = (dis[0] - dis[len(RX_coord)-1]) * 1000

print("Theoretische Werte von TX bei:", SIGN_coord[0], SIGN_coord[1])
doa_meters = [delta_x[0], delta_x[1], delta_x[2], delta_x[3], delta_x[4]]
doa_meters

('Theoretische Werte von TX bei:', 53.0915631, 8.7901959)


[8.5100921738394675,
 -1941.2306651870251,
 3062.7611553420556,
 -1955.1085994187492,
 -825.06801708987871]

### Transformation Kartesische - Geograpfische Koordinaten

In [9]:
def  xy2latlong_kl( x, y ):

    ref_lat = 53.0792962                                            #Längengrad Waller Fernsehturm
    ref_long = 8.8016937                                           #Breitengrad
    earth_radius = 40074                                      #Erdumfang

    lat = (y * 360 / earth_radius) + ref_lat;
    lon = ( (x*360) / (earth_radius * np.cos(ref_lat*np.pi/180)) ) + ref_long;

    return lat, lon

### Transformation Winkel in vielfache von pi 

In [10]:
def wrap2pi(angle1):
    wrapped_angle = angle1;
    
    if (angle1 > np.pi):
        wrapped_angle = -np.pi + (angle1-np.pi)
    
    if (angle1 < -np.pi):
        wrapped_angle =  np.pi + (angle1+np.pi)
    
    return wrapped_angle

### Transformation Geograpfische - Kartesiche Koordinaten

In [11]:
def latlong2xy_kl( lat, lon ):

    ref_lat = 53.0792962                                            #Längengrad Waller Fernsehturm
    ref_long = 8.8016937                                            #Breitengrad
    earth_radius = 40074                                      #Erdumfang

    y = (lat - ref_lat)/360 * earth_radius
    x = (lon - ref_long)/360 * np.cos(ref_lat*np.pi/180) * earth_radius

    return x, y

In [12]:
def dist_latlong_kl( lat1, long1, lat2, long2 ):

    [x1, y1] = latlong2xy_kl( lat1, long1 )
    [x2, y2] = latlong2xy_kl( lat2, long2 )

    dist = 1000 * np.sqrt( (x1-x2)**2 + (y1-y2)**2 )
    return dist

### Hyperbeln berechnen

In [13]:
#Funktion zur Erstellung der Hyperbeln

def gen_hyperbola( doa_meters, RX_coord,rx_1,rx_2,k):    
    
    if k < len(RX_coord)-1:
        [rx1_x, rx1_y] = latlong2xy_kl(RX_coord[k,0], RX_coord[k,1])
        [rx2_x, rx2_y] = latlong2xy_kl(RX_coord[k+1,0], RX_coord[k+1,1])
    if k == len(RX_coord)-1:
        [rx1_x, rx1_y] = latlong2xy_kl(RX_coord[0,0], RX_coord[0,1])
        [rx2_x, rx2_y] = latlong2xy_kl(RX_coord[len(RX_coord)-1,0], RX_coord[len(RX_coord)-1,1])

    
    print(rx1_x,rx2_x)
    rx_x_dist = rx2_x - rx1_x
    rx_y_dist = rx2_y - rx1_y

    dist_12 = abs (complex(rx_x_dist, rx_y_dist))
    angle_12 = np.angle (complex(rx_x_dist, rx_y_dist))

    hyp_x =[]
    hyp_y =[]

    hyp_x_leg1 =[]
    hyp_y_leg1 =[]
    hyp_x_leg2 =[]
    hyp_y_leg2 =[]
    hyp_point_counter = 0
    
    if abs(float(doa_meters)/1000) > dist_12:
        print("<strong>TODA delay (" ,str(doa_meters), " meters) larger than RX distance (", str(1000*  dist_12), " meters) -> no solution possible")
        doa_meters = np.sign(doa_meters) * 0.995 * dist_12 * 1000
        print("ATTENTION: Correcting TODA delay to 0.995 * RX distance (maximum possible value) = ", str(0.995*doa_meters) )
   
    if abs(float(doa_meters)/1000) <= dist_12:

        for r_1 in np.arange(0,10,0.05):
            r_2 = r_1 - float(doa_meters)/1000;
            #print("r_1 = ", str(r_1), ", r_2 = ", str(r_2))
            if ((r_2 + r_1) > dist_12):
                
                acos_argument = (r_2**2 - r_1**2 - dist_12**2) / (-2*r_1*dist_12)
                
                if (acos_argument >= -1) and (acos_argument <= +1):
                
                    
                    hyp_point_counter = hyp_point_counter + 1
                    hyp_angle = np.arccos(acos_argument)
                    abs_angle1 = wrap2pi(angle_12 + hyp_angle)
                    hyp_x_leg1.append(rx1_x + r_1 * np.cos(abs_angle1))
                    hyp_y_leg1.append(rx1_y + r_1 * np.sin(abs_angle1))

                    abs_angle2 = wrap2pi(angle_12 - hyp_angle)
                    hyp_x_leg2.append(rx1_x + r_1 * np.cos(abs_angle2))
                    hyp_y_leg2.append(rx1_y + r_1 * np.sin(abs_angle2))
                
                else:
                    print("acos argument ", str(acos_argument))
       
    else:
        print("TODA delay larger than RX distance -> no solution possible")
    
    if (hyp_point_counter == 0):
        print("Hyperbola could not be constructed")
    
    hyp_x_leg1= np.asarray(hyp_x_leg1)
    hyp_y_leg1= np.asarray(hyp_y_leg1)
    hyp_x_leg2= np.asarray(hyp_x_leg2)
    hyp_y_leg2= np.asarray(hyp_y_leg2)
    
    
    hyp_x = np.concatenate(((np.flipud(hyp_x_leg1)), hyp_x_leg2))
    hyp_y = np.concatenate(((np.flipud(hyp_y_leg1)), hyp_y_leg2))
    
    hyp_points = 2* hyp_point_counter

    points_lat = np.zeros(hyp_points)
    points_long = np.zeros(hyp_points)
    
    
    for ii in range(1, hyp_points):
        [points_lat[ii], points_long[ii]] = xy2latlong_kl(hyp_x[ii], hyp_y[ii])
       
        #print(xy2latlong_kl(hyp_x[ii], hyp_y[ii]))
        
    print("Hyperbola with totally ", str(hyp_points), " points generated.")
    return points_lat, points_long 

### OSM Karte mit Hyperbeln und Markierungen zeichnen

In [14]:
myMap= Map(center=[53.0792962, 8.8016937], zoom=12)               #Zeichne und skaliere Karte
dc=DrawControl(circle={'shapeOptions':{'color':'#0000FF'}})

marker_fernseh= Circle(location=[REF_coord[0],REF_coord[1]],radius=5)      #Markiere die Referenzposition
myMap.add_layer(marker_fernseh)                                 

marker_signal = Circle(location=[SIGN_coord[0],SIGN_coord[1]],radius=1,color='#cc0000',)
myMap.add_layer(marker_signal)

colors=['#cc0000', '#00cc00', '#0000cc','#cc00cc','#00cccc','#cc6600','#5d8aa8','#9966cc','#fdee00','#800020']

for i in range(0, len(RX_coord)):                                 #Markiere Empfangsstationen
    marker = Marker(location=[RX_coord[i,0], RX_coord[i,1]])
    myMap.add_layer(marker)

for k in range(0, len(RX_coord)):                                 #Aufrufen der Hyperbelfunktion
    for m in range(0, len(RX_coord)):                             #Übergabewerte festlegen
        if k < len(RX_coord) - 1:
            rx_1 = k
            rx_2 = k+1
        if k == len(RX_coord) - 1:
            rx_1 = 0
            rx_2 = len(RX_coord) - 1
        #print(rx_1,rx_2)   

    [x, y] = gen_hyperbola(doa_meters[k], RX_coord, rx_1, rx_2, k)#Rufe Funktion zum Hyperbeln generieren auf
    x_list, y_list = x.tolist(), y.tolist()                       #Umformung Arrays in Listen
    hyp_list = np.zeros((len(x_list), 2))                         #Nullmatrix generieren

    for j in range(1, len(x_list)):                               #Hyperbelwerte in Matrix speichern
        hyp_list[j, 0] = x[j]
        hyp_list[j, 1] = y[j]

    hyp_list[0, 0] = hyp_list[1, 0]                               #Erste Zeile weglassen da = 0
    hyp_list[0, 1] = hyp_list[1, 1]
    
    hyp = Polyline(locations=hyp_list.tolist())                   #Hyperbel als "polyline" generieren
 
    hyp.color = colors[k]                                         #Farbgebung der Hyperbeln
    hyp.fill_color = colors[k]
   
    
    myMap.add_layer(hyp)                                          #Zeichne Hyperbeln
myMap.add_control(dc)
myMap

(-0.5265059761013775, -2.7702264025981238)
('Hyperbola with totally ', '328', ' points generated.')
(-2.7702264025981238, -1.2258882238432587)
('Hyperbola with totally ', '338', ' points generated.')
(-1.2258882238432587, 0.21640795184698486)
('Hyperbola with totally ', '254', ' points generated.')
(0.21640795184698486, -3.4312927474853536)
('Hyperbola with totally ', '362', ' points generated.')
(-0.5265059761013775, -3.4312927474853536)
('Hyperbola with totally ', '356', ' points generated.')


Map(center=[53.0792962, 8.8016937], controls=(DrawControl(circle={'shapeOptions': {'color': '#0000FF'}}, layer=FeatureGroup(), polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}),), layers=(TileLayer(options=[u'opacity', u'attribution', u'max_zoom', u'detect_retina', u'min_zoom', u'tile_size']), Circle(location=[53.0950487, 8.7925119], options=[u'opacity', u'fill_color', u'weight', u'class_name', u'dash_array', u'color', u'stroke', u'line_cap', u'clickable', u'fill_opacity', u'pointer_events', u'line_join', u'fill'], radius=5), Circle(color='#cc0000', location=[53.0915631, 8.7901959], options=[u'opacity', u'fill_color', u'weight', u'class_name', u'dash_array', u'color', u'stroke', u'line_cap', u'clickable', u'fill_opacity', u'pointer_events', u'line_join', u'fill'], radius=1), Marker(location=[53.07264, 8.7938200000000002], options=[u'opacity', u'rise_offset', u'title', u'z_index_offset', u'draggable', u'keyboard', u'clickable', u'alt', u'rise_on_hover']), Marker(location=[53.097636999999999, 8.7602659999999997], options=[u'opacity', u'rise_offset', u'title', u'z_index_offset', u'draggable', u'keyboard', u'clickable', u'alt', u'rise_on_hover']), Marker(location=[53.055365999999999, 8.7833609999999993], options=[u'opacity', u'rise_offset', u'title', u'z_index_offset', u'draggable', u'keyboard', u'clickable', u'alt', u'rise_on_hover']), Marker(location=[53.090755999999999, 8.8049300000000006], options=[u'opacity', u'rise_offset', u'title', u'z_index_offset', u'draggable', u'keyboard', u'clickable', u'alt', u'rise_on_hover']), Marker(location=[53.080219999999997, 8.7503799999999998], options=[u'opacity', u'rise_offset', u'title', u'z_index_offset', u'draggable', u'keyboard', u'clickable', u'alt', u'rise_on_hover']), Polyline(color='#cc0000', fill_color='#cc0000', locations=[[53.030420074291655, 8.663515463627679], [53.030420074291655, 8.663515463627679], [53.03070592389791, 8.664108436451341], [53.030991823117276, 8.664701511701317], [53.03127777273973, 8.665294691008516], [53.03156377357225, 8.665887976038952], [53.03184982643929, 8.666481368494708], [53.03213593218325, 8.667074870114911], [53.03242209166497, 8.667668482676767], [53.03270830576424, 8.668262207996614], [53.032994575380336, 8.668856047931014], [53.033280901432576, 8.66945000437788], [53.033567284860865, 8.670044079277652], [53.03385372662629, 8.670638274614497], [53.034140227711724, 8.671232592417564], [53.03442678912246, 8.671827034762272], [53.03471341188685, 8.672421603771658], [53.03500009705698, 8.673016301617759], [53.035286845709365, 8.67361113052304], [53.03557365894568, 8.674206092761894], [53.03586053789347, 8.674801190662171], [53.03614748370698, 8.67539642660678], [53.0364344975679, 8.67599180303534], [53.03672158068623, 8.676587322445892], [53.03700873430114, 8.677182987396682], [53.03729595968182, 8.677778800508], [53.03758325812848, 8.678374764464094], [53.037870630973245, 8.67897088201515], [53.03815807958119, 8.679567155979358], [53.03844560535135, 8.680163589245044], [53.03873320971782, 8.68076018477289], [53.03902089415086, 8.681356945598242], [53.03930866015804, 8.681953874833503], [53.03959650928549, 8.68255097567062], [53.03988444311909, 8.683148251383674], [53.04017246328582, 8.683745705331566], [53.04046057145511, 8.684343340960815], [53.04074876934022, 8.68494116180847], [53.041037058699736, 8.685539171505138], [53.0413254413391, 8.686137373778125], [53.04161391911216, 8.686735772454735], [53.041902493922876, 8.687334371465672], [53.042191167727005, 8.68793317484861], [53.042479942533916, 8.68853218675189], [53.04276882040847, 8.689131411438408], [53.04305780347294, 8.689730853289625], [53.043346893909096, 8.69033051680979], [53.043636093960295, 8.690930406630322], [53.04392540593372, 8.69153052751439], [53.04421483220268, 8.692130884361701], [53.04450437520903, 8.692731482213489], [53.044794037465735, 8.69333232625774], [53.04508382155947, 8.693933421834652], [53.04537373015339, 8.694534774442324], [53.04566376599004, 8.695136389